In [1]:
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model
from datetime import datetime, timedelta

In [2]:
import sys
sys.path.append('/home/cayena/Projeto_thefamily/TeacherScheduler/')

In [3]:
from streamlit_app.utils import transform_classes_dateframe,transform_teacher_dataframe,transform_alocation_dataframe,base_selection,expand_rows,replicate_row,clean_data
from streamlit_app.teacher_alocation import TeacherScheduler
from streamlit_app.validador import validador

2025-04-02 14:01:20.197 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 14:01:20.205 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 14:01:20.220 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 14:01:20.230 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
aulas_raw = pd.read_excel('/home/cayena/Downloads/BASE PARA ROTA 10.03.25(2).xlsx')

professores_raw = pd.read_excel('/home/cayena/Downloads/TESTE TEACHERS 07.03(3).xlsx')

/home/cayena/.local/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [5]:
df_result =transform_classes_dateframe(aulas_raw)
df_professor = transform_teacher_dataframe(professores_raw)

In [6]:
Ts = TeacherScheduler(df_result, df_professor)
base_alocada = Ts.schedule_teachers()
base_alocada

3791
Não foi possível encontrar uma solução ótima.


,professores_alocados,nome grupo


In [14]:
df_final,aulas_nao_alocadas = transform_alocation_dataframe(aulas_raw,base_alocada)
aulas_nao_alocadas

,nome grupo,horario,unidade,dias da semana,stage,livro,modalidade,grupo,n aulas,parag atual grupo,...,rescisao,permuta,bolsista,n_total_alunos,stage_formula,parag_final_formula,num_1_sempre,ultimo_professor,penultimo_professor,restricoes_professor


In [15]:
numero_aulas = df_final.groupby('teacher').agg({'n aulas':'sum'}).sort_values('n aulas',ascending=False).reset_index()
numero_aulas = pd.merge(numero_aulas,df_professor[['TEACHER','MEDIA']],left_on='teacher',right_on='TEACHER',how='left')

In [16]:
numero_aulas

,teacher,n aulas,TEACHER,MEDIA
0,MARIA,36.0,MARIA,24
1,RAPHAEL,24.0,RAPHAEL,24
2,BRUNO MORGILLO,23.0,BRUNO MORGILLO,24
3,TELMA,23.0,TELMA,12
4,MATTHEW,21.0,MATTHEW,12
5,MADDIE,19.0,MADDIE,20
6,BARBARA,18.0,BARBARA,16
7,DARLENE,17.0,DARLENE,16
8,SAMARA,15.0,SAMARA,11
9,MADISON,14.0,MADISON,16


In [16]:
import pandera as pa
import pandas as pd
from datetime import datetime
import streamlit as st

In [18]:
import pandera as pa
import pandas as pd
from datetime import datetime
import streamlit as st

schema_model = pa.DataFrameSchema({
    'unidade': pa.Column(str, nullable=False),
    'dias da semana': pa.Column(object, nullable=False),
    'stage': pa.Column(str, nullable=False),
    'livro': pa.Column(object, nullable=True),
    'nome grupo': pa.Column(object, unique=True, nullable=False),
    'horario': pa.Column(datetime, nullable=False),
    'modalidade': pa.Column(str, nullable=False),
    'grupo': pa.Column(str, nullable=False),
    'n aulas': pa.Column(int, nullable=True),
    'teacher': pa.Column(str, nullable=False)    
})

df_exemplo = pd.read_excel(r"C:\Users\Erione Technologies\Downloads\CARLOS VIADO 2.xlsx")

st.title("Validação de Dados")

try:
    schema_model.validate(df_exemplo, lazy=True)
    st.success("✅ Os dados estão válidos e conforme o esquema definido!")
except pa.errors.SchemaErrors as exc:
    st.error("❌ Foram encontrados erros na validação dos dados:")
    
    # Acessando o objeto de falhas corretamente
    failure_cases = exc.failure_cases  # DataFrame com casos de falha
    schema_errors = exc.schema  # Schema que falhou
    
    # Mostrar resumo dos erros
    with st.expander("📊 Resumo dos erros"):
        st.write(f"Número total de erros: {len(failure_cases)}")
        st.write("Principais problemas encontrados:")
        
        # Agrupar erros por coluna e tipo
        error_summary = failure_cases.groupby(['column', 'check']) \
                                   .size() \
                                   .reset_index(name='count')
        st.dataframe(error_summary)
    
    # Mostrar detalhes dos erros
    with st.expander("🔍 Detalhes dos erros"):
        st.write("### Casos específicos que falharam:")
        st.dataframe(failure_cases)
    
    # Sugerir ações corretivas
    st.warning("📌 Ações recomendadas:")
    
    # Verificar erros comuns e sugerir soluções específicas
    if any(failure_cases['check'] == "not_nullable"):
        st.write("- Existem valores nulos em colunas que não os permitem")
    
    if any(failure_cases['check'] == "dtype"):
        st.write("- Existem valores com tipos incorretos em algumas colunas")
    
    if any(failure_cases['check'] == "unique"):
        st.write("- Existem valores duplicados em colunas que devem ser únicas")
    
    st.write("- Verifique se todas as colunas obrigatórias estão presentes")
    st.write("- Confira os formatos de data/horário se aplicável")

c:\Users\Erione Technologies\Documents\GitHub\TeacherScheduler\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
2025-04-28 16:46:10.985 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-28 16:46:10.985 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-28 16:46:11.006 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-28 16:46:11.007 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-28 16:46:11.008 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-28 16:46:11.009 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-28 16:46:11.010 Thread 'Ma